In [62]:
import csv
import numpy as np
import librosa
import librosa.display
import wave
import os
import matplotlib.pyplot as plt
import multiprocessing 

In [63]:
SOURCE_FOLDER_TRAIN= '../AudioData/audio-cats-and-dogs/cats_dogs/train/'
OUTPUT_FOLDER_TRAIN = '../GeneratedData/audio-cats-and-dogs/cats_dogs/train/'

In [64]:
SOURCE_FOLDER_TEST= '../AudioData/audio-cats-and-dogs/cats_dogs/test/'
OUTPUT_FOLDER_TEST = '../GeneratedData/audio-cats-and-dogs/cats_dogs/test/'

In [80]:
### Image output resolution
RESOLUTION = 224
### Spectograph resolution
SAMPLE_RATE = 16000
N_FFT = 1024
HOP_LENGTH = 128
N_MELS = RESOLUTION
FMIN = 60
FMAX = SAMPLE_RATE / 2 

In [81]:
def log_mel_spec_tfm(src_path, dst_path):
    print(src_path)
    fname=src_path.split('/')[-1];
    x=[]
    sample_rate=0
    try:
        channels ,sample_rate_in = librosa.load(src_path,mono=False)
    except:
        print(fname+" Could not be computed!")
        return
    if(channels.ndim==1):
        channels= channels.reshape((1,(len(channels))))
    channelcounter=0
    for x in channels:
        channelcounter+=1
        audio_data = librosa.resample(x, sample_rate_in, SAMPLE_RATE)
        mel_spec_power = librosa.feature.melspectrogram(x, sr=SAMPLE_RATE, n_fft=N_FFT, 
                                                        hop_length=HOP_LENGTH, 
                                                        n_mels=N_MELS, power=2.0,
                                                       fmin=FMIN,fmax=FMAX)
        mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
        width = np.shape(mel_spec_db)[1]
        if width >= N_MELS:
            for i in range(0,int(np.floor(width/N_MELS))):
                out= np.zeros(shape=(RESOLUTION,RESOLUTION))
                if((i+1)*N_MELS<=width):
                    out = mel_spec_db[:,(i)*N_MELS:(i+1)*N_MELS]
                    im = plt.imshow(out)
                    colors = im.cmap(im.norm(out))
                    data = colors.astype(np.float64) / np.max(colors) # normalize the data to 0 - 1
                    data = 255 * data # Now scale by 255
                    img = data.astype(np.uint8)
                    out = img[:,:,0:3] 
                plt.imsave(os.path.join(dst_path,(fname.replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+'-ch-'+str(channelcounter)+ '.png')), out) 

In [82]:
def ComputeSpectrograms(IN_FOLDER,OUT_FOLDER):
    SourceFoldersLabels = [f.path for f in os.scandir(IN_FOLDER) if f.is_dir()]
    for path in SourceFoldersLabels:
        FileList = [f.path for f in os.scandir(path) if f.is_file() and (f.name.endswith(".wav") or f.name.endswith(".m4a") or f.name.endswith(".mp3"))]
        Label = path.split('/')[-1]
        outFolder = os.path.join(OUT_FOLDER,Label)
        if not os.path.exists(outFolder):
            os.makedirs(outFolder)
        with multiprocessing.Pool(4) as p:
            p.starmap(log_mel_spec_tfm, [(f,outFolder) for f in FileList])
        print("Finished this pool. Now next Folder!")
    print("Done generating spectra!")  

In [84]:
ComputeSpectrograms(SOURCE_FOLDER_TRAIN,OUTPUT_FOLDER_TRAIN)
ComputeSpectrograms(SOURCE_FOLDER_TEST,OUTPUT_FOLDER_TEST)

../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_48.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_131.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_99.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_71.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_119.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_118.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_123.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_126.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_108.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_134.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_124.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_137.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_121.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_122.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_130.wav
../AudioData/audio-cats-and-

../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_58.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_29.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_12.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_98.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_13.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_66.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_72.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_67.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_17.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_14.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_20.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_22.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_16.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking

../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_38.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_1.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_21.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_26.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_42.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_35.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_112.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_32.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_0.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_57.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_18.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_43.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_106.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/dog/dog_barking_69.wav
../Aud

In [29]:
ComputeSpectrograms("../AudioData/urban-sound-classification/train/Train/","../GeneratedData/urban-sound-classification/train/Train/")

../AudioData/urban-sound-classification/train/Train/siren/850.wav
../AudioData/urban-sound-classification/train/Train/siren/4100.wav
../AudioData/urban-sound-classification/train/Train/siren/2002.wav
../AudioData/urban-sound-classification/train/Train/siren/7829.wav


/Users/dg/anaconda3/envs/Workshop/lib/python3.6/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/Users/dg/anaconda3/envs/Workshop/lib/python3.6/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/Users/dg/anaconda3/envs/Workshop/lib/python3.6/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/Users/dg/anaconda3/envs/Workshop/lib

../AudioData/urban-sound-classification/train/Train/siren/4473.wav
../AudioData/urban-sound-classification/train/Train/siren/1294.wav
../AudioData/urban-sound-classification/train/Train/siren/8706.wav
../AudioData/urban-sound-classification/train/Train/siren/2601.wav
../AudioData/urban-sound-classification/train/Train/siren/6270.wav
../AudioData/urban-sound-classification/train/Train/siren/663.wav
../AudioData/urban-sound-classification/train/Train/siren/728.wav
../AudioData/urban-sound-classification/train/Train/siren/249.wav
../AudioData/urban-sound-classification/train/Train/siren/6516.wav
../AudioData/urban-sound-classification/train/Train/siren/5637.wav
../AudioData/urban-sound-classification/train/Train/siren/922.wav
../AudioData/urban-sound-classification/train/Train/siren/2198.wav
../AudioData/urban-sound-classification/train/Train/siren/1251.wav
../AudioData/urban-sound-classification/train/Train/siren/3520.wav
../AudioData/urban-sound-classification/train/Train/siren/3721.wav

/Users/dg/anaconda3/envs/Workshop/lib/python3.6/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


../AudioData/urban-sound-classification/train/Train/siren/7975.wav
../AudioData/urban-sound-classification/train/Train/siren/1899.wav
../AudioData/urban-sound-classification/train/Train/siren/6880.wav
../AudioData/urban-sound-classification/train/Train/siren/2363.wav
../AudioData/urban-sound-classification/train/Train/siren/3087.wav
../AudioData/urban-sound-classification/train/Train/siren/8665.wav
../AudioData/urban-sound-classification/train/Train/siren/8675.wav
../AudioData/urban-sound-classification/train/Train/siren/2439.wav
../AudioData/urban-sound-classification/train/Train/siren/1848.wav
../AudioData/urban-sound-classification/train/Train/siren/8670.wav
../AudioData/urban-sound-classification/train/Train/siren/6300.wav
../AudioData/urban-sound-classification/train/Train/siren/467.wav
../AudioData/urban-sound-classification/train/Train/siren/4918.wav
../AudioData/urban-sound-classification/train/Train/siren/6661.wav
../AudioData/urban-sound-classification/train/Train/siren/2614.

...

/Users/dg/anaconda3/envs/Workshop/lib/python3.6/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


../AudioData/urban-sound-classification/train/Train/siren/8704.wav
../AudioData/urban-sound-classification/train/Train/siren/843.wav
../AudioData/urban-sound-classification/train/Train/siren/2955.wav
../AudioData/urban-sound-classification/train/Train/siren/6505.wav
../AudioData/urban-sound-classification/train/Train/siren/1719.wav
../AudioData/urban-sound-classification/train/Train/siren/5234.wav
../AudioData/urban-sound-classification/train/Train/siren/8277.wav
../AudioData/urban-sound-classification/train/Train/siren/3469.wav
../AudioData/urban-sound-classification/train/Train/siren/8715.wav
../AudioData/urban-sound-classification/train/Train/siren/2573.wav
../AudioData/urban-sound-classification/train/Train/siren/7150.wav
../AudioData/urban-sound-classification/train/Train/siren/3441.wav
../AudioData/urban-sound-classification/train/Train/siren/3656.wav
../AudioData/urban-sound-classification/train/Train/siren/2205.wav
../AudioData/urban-sound-classification/train/Train/siren/6077.

../AudioData/urban-sound-classification/train/Train/siren/5709.wav
../AudioData/urban-sound-classification/train/Train/siren/7681.wav
../AudioData/urban-sound-classification/train/Train/siren/2659.wav
../AudioData/urban-sound-classification/train/Train/siren/1236.wav
../AudioData/urban-sound-classification/train/Train/siren/3344.wav
../AudioData/urban-sound-classification/train/Train/siren/2921.wav
../AudioData/urban-sound-classification/train/Train/siren/6417.wav
../AudioData/urban-sound-classification/train/Train/siren/211.wav
../AudioData/urban-sound-classification/train/Train/siren/3813.wav
../AudioData/urban-sound-classification/train/Train/siren/6548.wav
../AudioData/urban-sound-classification/train/Train/siren/60.wav
../AudioData/urban-sound-classification/train/Train/siren/3186.wav
../AudioData/urban-sound-classification/train/Train/siren/3191.wav
../AudioData/urban-sound-classification/train/Train/siren/943.wav
../AudioData/urban-sound-classification/train/Train/siren/5095.wav

../AudioData/urban-sound-classification/train/Train/siren/391.wav
../AudioData/urban-sound-classification/train/Train/siren/3987.wav
../AudioData/urban-sound-classification/train/Train/siren/5936.wav
../AudioData/urban-sound-classification/train/Train/siren/6811.wav
../AudioData/urban-sound-classification/train/Train/siren/4027.wav
../AudioData/urban-sound-classification/train/Train/siren/8187.wav
../AudioData/urban-sound-classification/train/Train/siren/5063.wav
../AudioData/urban-sound-classification/train/Train/siren/427.wav
../AudioData/urban-sound-classification/train/Train/siren/6156.wav
../AudioData/urban-sound-classification/train/Train/siren/6804.wav
../AudioData/urban-sound-classification/train/Train/siren/3978.wav
../AudioData/urban-sound-classification/train/Train/siren/8190.wav
../AudioData/urban-sound-classification/train/Train/siren/8547.wav
../AudioData/urban-sound-classification/train/Train/siren/1831.wav
../AudioData/urban-sound-classification/train/Train/siren/6817.w

KeyboardInterrupt: 

In [ ]:
'''
Unused function
'''
def generateOutputFolder(overwrite, path,labels_local):
    if not os.path.exists(path):
        #if(overwrite):
            #TODO actually delete the folder
        os.makedirs(path)
    TemporaryDict={}
    for i in labels_local:
        LabeldDirectory = os.path.join(path,i)
        if not os.path.exists(LabeldDirectory):
            os.makedirs(LabeldDirectory)
        TemporaryDict[i]=LabeldDirectory
    return TemporaryDict